<a href="https://colab.research.google.com/github/LewisRSP/AsistenteVirtual/blob/main/RT1/asesorvirtualconia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conexión Bluetooth
def connect_bluetooth(COM5, 9600):
    """
    Conecta al Arduino a través de Bluetooth.

    Args:
        port: Puerto serial del dispositivo Bluetooth.
        baudrate: Velocidad de transmisión de datos.

    Returns:
        Objeto serial si la conexión es exitosa, None en caso de error.
    """
    try:
        bt = serial.Serial(port, baudrate, timeout=1)
        print(f"Connected to Arduino on {port}")
        return bt
    except Exception as e:
        print(f"Error connecting to Bluetooth: {e}")
        return None

In [ ]:
# Conexión a MySQL
def connect_database(localhost, root, Lewis110578, mantenimiento):
    """
    Conecta a la base de datos MySQL.

    Args:
        host: Dirección del servidor MySQL.
        user: Nombre de usuario de la base de datos.
        password: Contraseña del usuario.
        database: Nombre de la base de datos.

    Returns:
        Objeto de conexión a la base de datos si la conexión es exitosa, None en caso de error.
    """
    try:
        db = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        print("Connected to MySQL database")
        return db
    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

In [ ]:
# Procesamiento de comandos usando NLP
def process_command(command):
    """
    Utiliza un modelo de NLP para clasificar el comando.

    Args:
        command: Comando recibido del Arduino.

    Returns:
        Diccionario con la etiqueta y la confianza del resultado.
    """
    nlp_model = pipeline("text-classification", model="distilbert-base-uncased")
    result = nlp_model(command)
    return result[0]

In [ ]:
# Registrar información en MySQL
def log_to_database(cursor, command, response):
    """
    Registra el comando y la respuesta en la base de datos MySQL.

    Args:
        cursor: Objeto cursor de la conexión a la base de datos.
        command: Comando recibido del Arduino.
        response: Respuesta procesada por el modelo de NLP.
    """
    try:
        query = "INSERT INTO maintenance_logs (command, response, timestamp) VALUES (%s, %s, NOW())"
        cursor.execute(query, (command, response))
        print("Data logged successfully")
    except Exception as e:
        print(f"Error logging data: {e}")

In [ ]:
# Main
if __name__ == "__main__":
    # Conexión Bluetooth
    arduino_bt = connect_bluetooth("COM5", 9600)  # Cambiar el puerto según el dispositivo

    # Conexión MySQL
    db_connection = connect_database("localhost", "root", "password", "maintenance_db")
    if db_connection:
        cursor = db_connection.cursor()

    # Procesamiento continuo
    while True:
        if arduino_bt and arduino_bt.in_waiting > 0:
            # Leer mensaje de Arduino
            command = arduino_bt.readline().decode().strip()
            print(f"Command received: {command}")

            # Procesar el comando
            nlp_response = process_command(command)
            response = nlp_response['label']
            confidence = nlp_response['score']

            # Responder al Arduino
            response_message = f"{response} ({confidence:.2f})"
            arduino_bt.write(response_message.encode())
            print(f"Response sent: {response_message}")

            # Loguear en MySQL
            if db_connection:
                log_to_database(cursor, command, response)
                db_connection.commit()

        time.sleep(1)